### HyperParameter Tau 를 이용한 Tumor Segmentation

Red Pixel - (Greed Pixel + Blue Pixel) /2 > tau 에 해당하는 부분을 다른 색으로 채워서 출력

In [10]:
! pip install pandas numpy opencv-python matplotlib 
! pip install pillow 
! pip install tqdm

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import cv2 
import os 
from tqdm import tqdm

from PIL import Image 
IMAGE_PATH = ["SF18_00841_N.jpg", "SS17-77649_N.jpg", "SS18-19871_N.jpg", "SS21-26050_P.jpg", "SS22-26507_P.jpg", "SS22-65597_P.jpg"]

#### Check whether Images are Loading 

In [3]:
N_00841 = Image.open(IMAGE_PATH[0])
N_77649 = Image.open(IMAGE_PATH[1])
N_19871 = Image.open(IMAGE_PATH[2])
P_26050 = Image.open(IMAGE_PATH[3])
P_26507 = Image.open(IMAGE_PATH[4])
P_65596 = Image.open(IMAGE_PATH[5])

- 각 이미지의 크기와 특징들을 출력

In [4]:
def info_image(image_path):
    print("************************************") 
    for path in image_path:
        img = Image.open(path)
        print(f"이미지의 이름 : {path}") 
        print(f"이미지의 형식(formatting) : {img.format}")
        print(f"이미지의 용량(size) : {img.size}")
        print(f"이미지의 색상모드 :{img.mode}")
        print(f"이미지의 Width : {img.width}")
        print(f"이미지의 Height : {img.height}")

        print("************************************") 


In [14]:
info_image(IMAGE_PATH)

************************************
이미지의 이름 : SF18_00841_N.jpg
이미지의 형식(formatting) : JPEG
이미지의 용량(size) : (7817, 5536)
이미지의 색상모드 :RGB
이미지의 Width : 7817
이미지의 Height : 5536
************************************
이미지의 이름 : SS17-77649_N.jpg
이미지의 형식(formatting) : JPEG
이미지의 용량(size) : (6498, 5536)
이미지의 색상모드 :RGB
이미지의 Width : 6498
이미지의 Height : 5536
************************************
이미지의 이름 : SS18-19871_N.jpg
이미지의 형식(formatting) : JPEG
이미지의 용량(size) : (7817, 5536)
이미지의 색상모드 :RGB
이미지의 Width : 7817
이미지의 Height : 5536
************************************
이미지의 이름 : SS21-26050_P.jpg
이미지의 형식(formatting) : JPEG
이미지의 용량(size) : (9912, 5536)
이미지의 색상모드 :RGB
이미지의 Width : 9912
이미지의 Height : 5536
************************************
이미지의 이름 : SS22-26507_P.jpg
이미지의 형식(formatting) : JPEG
이미지의 용량(size) : (8840, 5302)
이미지의 색상모드 :RGB
이미지의 Width : 8840
이미지의 Height : 5302
************************************
이미지의 이름 : SS22-65597_P.jpg
이미지의 형식(formatting) : JPEG
이미지의 용량(size) : (4773, 3928)
이미지의 색상모드 :RGB
이미지의 

- 해당 Heatmap으로 만들어진 이미지는 원본 Svs file의 1/16 크키
- Patch size 가 32, 64 크기로 만들예정이므로, 2x2, 4x4 로 Jpg 영역을 확인해봐야 할 거 같음

In [1]:
# SF18_00841_N.jpg 을 기준으로 시작해보자
# 4x4 로 만든다고 가정하고, Row, Col은 해당 Patch Size로 짤릴 수 있을 만큼

def make_segmentation_Image(image_path, tau, patch_size=2):
    for path in tqdm(image_path):
        print(f"Current Processins Image : {path}, Tau : {tau}, Patch Size : {patch_size}")
        image = Image.open(path)
        # segment_image = image.copy()
        image_width, image_height = image.width, image.height
        color = (255, 255, 255)
        segment_image = Image.new("RGB", (image_width, image_height), color) 
        # Patch로 만들 때, 해당 Patch Size로 잘라야 하므로, image_size //patch_size == 0이 되도록 설정
        MAX_WIDTH = image_width
        MAX_HEIGHT = image_height

        def is_verify(patch, tau, patch_size):
            sum_r, sum_g, sum_b = 0, 0, 0
            for pixel in patch.getdata():
                sum_r += pixel[0]
                sum_g += pixel[1]
                sum_b += pixel[2]
    
            if (sum_r - (sum_g + sum_b)/2) / (patch_size ** 2) > tau : 
                return True
            else:
                return False

        for h in range(0, MAX_HEIGHT - patch_size, patch_size):
            for w in range(0, MAX_WIDTH - patch_size, patch_size):
                # patch_x, patch_y, patch_h, patch _w : 패치 시작점 X, Y 좌표, 패치 높이, 너비
                # list(image.crop((patch_x, patch_y, patch_x+patch_w, patch_y+patch_h)).getdata())
                patch = image.crop((w, h, w + patch_size, h + patch_size))
                if is_verify(patch, tau, patch_size):
                    # 무슨 색으로 칠할지 고민즁 ... 보라색 : [139, 0, 255]
                    new_pixels = [(0, 0,255) for _ in range(patch_size)]
                    patch.putdata(new_pixels)

                    # 이미지에 해당 패치에 해당하는 부분의 픽셀값을 넣기
                    segment_image.paste(patch, (w, h))
        # 만들고자 하는 Segment 폴더 생성
        if not os.path.exists(f"segment_image/patch_{patch_size}_0316"):
            os.mkdir(f"segment_image/patch_{patch_size}_0316")
        print(f"Saving segment_image/patch_{patch_size}_0316/{path[:-4]}_tau={tau}.jpg ...")
        segment_image.save(f"segment_image/patch_{patch_size}_0316/{path[:-4]}_tau={tau}.jpg")
        print("Save Complete !!")

### Patch Size : 2 

In [5]:
Tau = [i for i in range(99,105)]
for tau in tqdm(Tau):
    make_segmentation_Image(IMAGE_PATH, tau)

  0%|          | 0/6 [00:00<?, ?it/s]

Current Processins Image : SF18_00841_N.jpg, Tau : 99, Patch Size : 2
Saving segment_image/patch_2_0316/SF18_00841_N_tau=99.jpg ...


Save Complete !!
Current Processins Image : SS17-77649_N.jpg, Tau : 99, Patch Size : 2
Saving segment_image/patch_2_0316/SS17-77649_N_tau=99.jpg ...


Save Complete !!
Current Processins Image : SS18-19871_N.jpg, Tau : 99, Patch Size : 2


  0%|          | 0/6 [02:08<?, ?it/s]


KeyboardInterrupt: 

### Patch Size : 4

In [ ]:
Tau = [i for i in range(80, 155)]
for tau in tqdm(Tau):
    make_segmentation_Image(IMAGE_PATH, tau, patch_size=4)